# Computation of OEIS sequences A334254 and A334255 for n=6 and checking their values for n=3,4,5. 

# 1. Main code.

In [1]:
%load_ext Cython

In [2]:
%%cython --annotate
from math import log 

cpdef ProcessRT(t,int i,int l,int nn1):
    """This function recursively extends an input tuple of integers, which represents 
    a set of sets (more precisily, a formal context), by adding to it an integer i (a new set).
    The length of a terminal tuple is constrained by l, while nn1 should be passed as 2**n-1. 
    The function checks whether the new tuple forms a unique closure system and returns the dictionary of the first elements of
    processed tuples as keys and values 1 for clousre system with T1 propery and 0 otherwise."""
    
    count={}
    t=t[:]
    cdef int P
    cdef int lent=len(t)

    
    #checking whether the context is reducible
    for j in range(lent-1,-1,-1):
        P=i
        #print("t[j],i",t[j],i,t[j]&i,)
        tji=t[j]&i
        if tji in t[0:j]:
            return count
            
        if tji==t[j]:
            #print("t[j],i",t[j],i,t[j]&i,)
            for h in range(j+1,lent):
                if t[j]&t[h]==t[j]:
                    P=P&t[h]
                    if P==t[j]:                        
                        return count
    t.append(i)

    #checking T1 property
    if Check(t,nn1):
        count[(lent+1,t[0])]=1
    else:
        count[(lent+1,t[0])]=0

    #recursive call
    cdef unsigned long long r
    #print(l)
    if l!=0:
        r=0
        for k in range(i+1,nn1):
            #print("Process(t,k,l-1,tuples,nn1)")
            #print("t",t,"k",k)
            ucount=ProcessRT(t,k,l-1,nn1)
            for key in ucount:
                if key in count:
                    count[key]+=ucount[key]
                else:
                    count[key]=ucount[key]
            #if r<0: print("r",r)
        return count
    else: 
        #print(tuples)
        #tuples.append(tuple(t))
       
        
        print("Branch process",t[0])
        return count

    
def isClosed(t,g):
    """Checks whether g is closed in t."""
    cont=[]
    for h in t:
        if g&h==g:
            cont.append(h)
    if len(cont)>0:
        res=cont[0]
        for h in cont:
            res=h&res
        return res==g
    else:
        return g==0

    
def Check(t,nn1):
    """Checks whether T1 is fulfilled for t."""
    n=int(log(nn1+1,2))
    members=[0]
    members.extend([2**i for i in range(n)])
    for m in members:
        if not isClosed(t,m):
            return False
    return True

In [3]:
def merge(r):
    """This is a value collector for the resulting list of dictionaries
    """
    merged={}
    for dic in r:
        for key in dic:
            if key in merged:
                merged[key]+=dic[key]
            else:
                merged[key]=dic[key]
    return merged

# 2. Counting A334254 for n=6.

In [28]:
%%time



from multiprocess import Pool
from multiprocess import cpu_count


n=6

#computing A334254 for n=6 by levels
if __name__ == "__main__":
    pool = Pool(cpu_count())
    
    nn1=2**n-1

    lt=[]
    lk=[]

    count={}

    
    for t in range(nn1):
 
        for i in range(t+1,nn1):
            lt.append([t])
            lk.append(i)
            
    ln_2=[nn1-1]* len(lt)
    lnn1=[nn1]* len(lt)
           
   
     
    print(list(zip(lt,lk,ln_2,lnn1)))

    
    #parallel execution of ProcessRT function
    res6 = pool.starmap_async(ProcessRT, zip(lt,lk,ln_2,lnn1))
    print(res6.get())# print the list of resulting dictionaries
    print(sum(merge(res6.get()).values()))#print the number of closure systems w.r.t. T1
    pool.close()  # 'TERM'
    pool.join()   # 'KILL'

[([0], 1, 62, 63), ([0], 2, 62, 63), ([0], 3, 62, 63), ([0], 4, 62, 63), ([0], 5, 62, 63), ([0], 6, 62, 63), ([0], 7, 62, 63), ([0], 8, 62, 63), ([0], 9, 62, 63), ([0], 10, 62, 63), ([0], 11, 62, 63), ([0], 12, 62, 63), ([0], 13, 62, 63), ([0], 14, 62, 63), ([0], 15, 62, 63), ([0], 16, 62, 63), ([0], 17, 62, 63), ([0], 18, 62, 63), ([0], 19, 62, 63), ([0], 20, 62, 63), ([0], 21, 62, 63), ([0], 22, 62, 63), ([0], 23, 62, 63), ([0], 24, 62, 63), ([0], 25, 62, 63), ([0], 26, 62, 63), ([0], 27, 62, 63), ([0], 28, 62, 63), ([0], 29, 62, 63), ([0], 30, 62, 63), ([0], 31, 62, 63), ([0], 32, 62, 63), ([0], 33, 62, 63), ([0], 34, 62, 63), ([0], 35, 62, 63), ([0], 36, 62, 63), ([0], 37, 62, 63), ([0], 38, 62, 63), ([0], 39, 62, 63), ([0], 40, 62, 63), ([0], 41, 62, 63), ([0], 42, 62, 63), ([0], 43, 62, 63), ([0], 44, 62, 63), ([0], 45, 62, 63), ([0], 46, 62, 63), ([0], 47, 62, 63), ([0], 48, 62, 63), ([0], 49, 62, 63), ([0], 50, 62, 63), ([0], 51, 62, 63), ([0], 52, 62, 63), ([0], 53, 62, 63), (

In [31]:
#saving the result to A334254_6.pickle file with pickle
import pickle
pikd = open("A334254_6.pickle", "wb")
pickle.dump(res6.get(), pikd)
pikd.close()

In [36]:
res6.get()

[{(2, 0): 0,
  (3, 0): 0,
  (4, 0): 0,
  (5, 0): 0,
  (6, 0): 0,
  (7, 0): 0,
  (8, 0): 0,
  (9, 0): 0},
 {(2, 0): 0,
  (3, 0): 0,
  (4, 0): 0,
  (5, 0): 0,
  (6, 0): 0,
  (7, 0): 0,
  (8, 0): 0,
  (9, 0): 0},
 {(2, 0): 0,
  (3, 0): 0,
  (4, 0): 0,
  (5, 0): 0,
  (6, 0): 0,
  (7, 0): 0,
  (8, 0): 0,
  (9, 0): 0,
  (10, 0): 0,
  (11, 0): 0,
  (12, 0): 0},
 {(2, 0): 0,
  (3, 0): 0,
  (4, 0): 0,
  (5, 0): 0,
  (6, 0): 0,
  (7, 0): 0,
  (8, 0): 0,
  (9, 0): 0},
 {(2, 0): 0,
  (3, 0): 0,
  (4, 0): 0,
  (5, 0): 0,
  (6, 0): 0,
  (7, 0): 0,
  (8, 0): 0,
  (9, 0): 0,
  (10, 0): 0,
  (11, 0): 0,
  (12, 0): 0},
 {(2, 0): 0,
  (3, 0): 0,
  (4, 0): 0,
  (5, 0): 0,
  (6, 0): 0,
  (7, 0): 0,
  (8, 0): 0,
  (9, 0): 0,
  (10, 0): 0,
  (11, 0): 0,
  (12, 0): 0},
 {(2, 0): 0,
  (3, 0): 0,
  (4, 0): 0,
  (5, 0): 0,
  (6, 0): 0,
  (7, 0): 0,
  (8, 0): 0,
  (9, 0): 0,
  (10, 0): 0,
  (11, 0): 0,
  (12, 0): 0,
  (13, 0): 0,
  (14, 0): 0},
 {(2, 0): 0,
  (3, 0): 0,
  (4, 0): 0,
  (5, 0): 0,
  (6, 0): 0,
  (7

In [38]:
sum(merge(res6.get()).values())# the computed value A334254 for n=6

66960965307

# 3. Checking A334254 for n=3, 4, 5.

In [5]:
%%time



from multiprocess import Pool
from multiprocess import cpu_count


n=5

#computing A334254 for n=5 by levels
if __name__ == "__main__":
    pool = Pool(cpu_count())
    
    nn1=2**n-1

    lt=[]
    lk=[]

    count={}

    
    for t in range(nn1):
 
        for i in range(t+1,nn1):
            lt.append([t])
            lk.append(i)
            
    ln_2=[nn1-1]* len(lt)
    lnn1=[nn1]* len(lt)
           
  
     
    print(list(zip(lt,lk,ln_2,lnn1)))

    
    #parallel execution of ProcessRT function
    res5 = pool.starmap_async(ProcessRT, zip(lt,lk,ln_2,lnn1))
    print(res5.get())# print the list of resulting dictionaries
    print(sum(merge(res5.get()).values()))#print the number of closure systems w.r.t. T1

    pool.close()  # 'TERM'
    pool.join()   # 'KILL'

[([0], 1, 30, 31), ([0], 2, 30, 31), ([0], 3, 30, 31), ([0], 4, 30, 31), ([0], 5, 30, 31), ([0], 6, 30, 31), ([0], 7, 30, 31), ([0], 8, 30, 31), ([0], 9, 30, 31), ([0], 10, 30, 31), ([0], 11, 30, 31), ([0], 12, 30, 31), ([0], 13, 30, 31), ([0], 14, 30, 31), ([0], 15, 30, 31), ([0], 16, 30, 31), ([0], 17, 30, 31), ([0], 18, 30, 31), ([0], 19, 30, 31), ([0], 20, 30, 31), ([0], 21, 30, 31), ([0], 22, 30, 31), ([0], 23, 30, 31), ([0], 24, 30, 31), ([0], 25, 30, 31), ([0], 26, 30, 31), ([0], 27, 30, 31), ([0], 28, 30, 31), ([0], 29, 30, 31), ([0], 30, 30, 31), ([1], 2, 30, 31), ([1], 3, 30, 31), ([1], 4, 30, 31), ([1], 5, 30, 31), ([1], 6, 30, 31), ([1], 7, 30, 31), ([1], 8, 30, 31), ([1], 9, 30, 31), ([1], 10, 30, 31), ([1], 11, 30, 31), ([1], 12, 30, 31), ([1], 13, 30, 31), ([1], 14, 30, 31), ([1], 15, 30, 31), ([1], 16, 30, 31), ([1], 17, 30, 31), ([1], 18, 30, 31), ([1], 19, 30, 31), ([1], 20, 30, 31), ([1], 21, 30, 31), ([1], 22, 30, 31), ([1], 23, 30, 31), ([1], 24, 30, 31), ([1], 25,

[{(2, 0): 0, (3, 0): 0, (4, 0): 0, (5, 0): 0, (6, 0): 0}, {(2, 0): 0, (3, 0): 0, (4, 0): 0, (5, 0): 0, (6, 0): 0}, {(2, 0): 0, (3, 0): 0, (4, 0): 0, (5, 0): 0, (6, 0): 0, (7, 0): 0, (8, 0): 0}, {(2, 0): 0, (3, 0): 0, (4, 0): 0, (5, 0): 0, (6, 0): 0}, {(2, 0): 0, (3, 0): 0, (4, 0): 0, (5, 0): 0, (6, 0): 0, (7, 0): 0, (8, 0): 0}, {(2, 0): 0, (3, 0): 0, (4, 0): 0, (5, 0): 0, (6, 0): 0, (7, 0): 0, (8, 0): 0}, {(2, 0): 0, (3, 0): 0, (4, 0): 0, (5, 0): 0, (6, 0): 0, (7, 0): 0, (8, 0): 0}, {(2, 0): 0, (3, 0): 0, (4, 0): 0, (5, 0): 0, (6, 0): 0}, {(2, 0): 0, (3, 0): 0, (4, 0): 0, (5, 0): 0, (6, 0): 0, (7, 0): 0, (8, 0): 0}, {(2, 0): 0, (3, 0): 0, (4, 0): 0, (5, 0): 0, (6, 0): 0, (7, 0): 0, (8, 0): 0}, {(2, 0): 0, (3, 0): 0, (4, 0): 0, (5, 0): 0, (6, 0): 0, (7, 0): 0, (8, 0): 0}, {(2, 0): 0, (3, 0): 0, (4, 0): 0, (5, 0): 0, (6, 0): 0, (7, 0): 0}, {(2, 0): 0, (3, 0): 0, (4, 0): 0, (5, 0): 0, (6, 0): 0, (7, 0): 0, (8, 0): 0}, {(2, 0): 0, (3, 0): 0, (4, 0): 0, (5, 0): 0, (6, 0): 0, (7, 0): 0}, {(2

In [9]:
sum(merge(res5.get()).values())# the computed value A334254 for n=5

702525

In [6]:
%%time



from multiprocess import Pool
from multiprocess import cpu_count


n=4

#computing A334254 for n=4 by levels
if __name__ == "__main__":
    pool = Pool(cpu_count())
    
    nn1=2**n-1

    lt=[]
    lk=[]

    count={}

    
    for t in range(nn1):
 
        for i in range(t+1,nn1):
            lt.append([t])
            lk.append(i)
            
    ln_2=[nn1-1]* len(lt)
    lnn1=[nn1]* len(lt)
           
  
     
    print(list(zip(lt,lk,ln_2,lnn1)))

    
    #parallel execution of ProcessRT function
    res4 = pool.starmap_async(ProcessRT, zip(lt,lk,ln_2,lnn1))
    print(res4.get())# print the list of resulting dictionaries
    print(sum(merge(res4.get()).values()))#print the number of closure systems w.r.t. T1

    pool.close()  # 'TERM'
    pool.join()   # 'KILL'

[([0], 1, 14, 15), ([0], 2, 14, 15), ([0], 3, 14, 15), ([0], 4, 14, 15), ([0], 5, 14, 15), ([0], 6, 14, 15), ([0], 7, 14, 15), ([0], 8, 14, 15), ([0], 9, 14, 15), ([0], 10, 14, 15), ([0], 11, 14, 15), ([0], 12, 14, 15), ([0], 13, 14, 15), ([0], 14, 14, 15), ([1], 2, 14, 15), ([1], 3, 14, 15), ([1], 4, 14, 15), ([1], 5, 14, 15), ([1], 6, 14, 15), ([1], 7, 14, 15), ([1], 8, 14, 15), ([1], 9, 14, 15), ([1], 10, 14, 15), ([1], 11, 14, 15), ([1], 12, 14, 15), ([1], 13, 14, 15), ([1], 14, 14, 15), ([2], 3, 14, 15), ([2], 4, 14, 15), ([2], 5, 14, 15), ([2], 6, 14, 15), ([2], 7, 14, 15), ([2], 8, 14, 15), ([2], 9, 14, 15), ([2], 10, 14, 15), ([2], 11, 14, 15), ([2], 12, 14, 15), ([2], 13, 14, 15), ([2], 14, 14, 15), ([3], 4, 14, 15), ([3], 5, 14, 15), ([3], 6, 14, 15), ([3], 7, 14, 15), ([3], 8, 14, 15), ([3], 9, 14, 15), ([3], 10, 14, 15), ([3], 11, 14, 15), ([3], 12, 14, 15), ([3], 13, 14, 15), ([3], 14, 14, 15), ([4], 5, 14, 15), ([4], 6, 14, 15), ([4], 7, 14, 15), ([4], 8, 14, 15), ([4], 9

In [11]:
sum(merge(res4.get()).values())# the computed value A334254 for n=4

545

In [8]:
%%time



from multiprocess import Pool
from multiprocess import cpu_count


n=3

#computing A334254 for n=3 by levels
if __name__ == "__main__":
    pool = Pool(cpu_count())
    
    nn1=2**n-1

    lt=[]
    lk=[]

    count={}

    
    for t in range(nn1):
 
        for i in range(t+1,nn1):
            lt.append([t])
            lk.append(i)
            
    ln_2=[nn1-1]* len(lt)
    lnn1=[nn1]* len(lt)
           
  
     
    print(list(zip(lt,lk,ln_2,lnn1)))

    
    #parallel execution of ProcessRT function
    res3 = pool.starmap_async(ProcessRT, zip(lt,lk,ln_2,lnn1))
    print(res3.get())# print the list of resulting dictionaries
    print(sum(merge(res3.get()).values()))#print the number of closure systems w.r.t. T1

    pool.close()  # 'TERM'
    pool.join()   # 'KILL'

[([0], 1, 6, 7), ([0], 2, 6, 7), ([0], 3, 6, 7), ([0], 4, 6, 7), ([0], 5, 6, 7), ([0], 6, 6, 7), ([1], 2, 6, 7), ([1], 3, 6, 7), ([1], 4, 6, 7), ([1], 5, 6, 7), ([1], 6, 6, 7), ([2], 3, 6, 7), ([2], 4, 6, 7), ([2], 5, 6, 7), ([2], 6, 6, 7), ([3], 4, 6, 7), ([3], 5, 6, 7), ([3], 6, 6, 7), ([4], 5, 6, 7), ([4], 6, 6, 7), ([5], 6, 6, 7)]
[{(2, 0): 0, (3, 0): 0}, {(2, 0): 0, (3, 0): 0}, {(2, 0): 0, (3, 0): 0}, {(2, 0): 0, (3, 0): 0}, {(2, 0): 0, (3, 0): 0}, {(2, 0): 0}, {(2, 1): 0, (3, 1): 1, (4, 1): 4}, {(2, 1): 0, (3, 1): 0, (4, 1): 1}, {(2, 1): 0, (3, 1): 0}, {(2, 1): 0, (3, 1): 0}, {(2, 1): 0}, {(2, 2): 0, (3, 2): 0, (4, 2): 1}, {(2, 2): 0, (3, 2): 0}, {(2, 2): 0, (3, 2): 0}, {(2, 2): 0}, {(2, 3): 0, (3, 3): 0}, {(2, 3): 0, (3, 3): 1}, {(2, 3): 0}, {(2, 4): 0}, {(2, 4): 0}, {(2, 5): 0}]
8
CPU times: user 21 ms, sys: 32 ms, total: 53.1 ms
Wall time: 57 ms


In [10]:
sum(merge(res3.get()).values())# the computed value A334254 for n=3

8